## PHASE 1 : BUSINESS UNDERSTANDING

# Problem statement

Identify which foods and lifecycle stages (farm, processing, transport, etc.) drive the largest environmental burdens, and recommend the highest-leverage actions (dietary swaps, sourcing policies, logistics/packaging changes) to lower total impact without undermining nutrition or cost.


# Stakeholders & decisions

- Policy & NGOs: dietary guidance, incentives for lower-impact foods, water-scarcity risk management. 

- Procurement & Retail: product mix, supplier selection, transport/packaging optimization.

- Producers/Farmers: practice changes (feed, fertilizer, irrigation efficiency).

- Consumers: informed swaps toward lower-impact alternatives.

# Success metrics (KPIs)

- GHG intensity (kg CO₂-eq per kg product; and optionally per 1000 kcal / per 100 g protein for fair comparisons across food types).

- Water footprint (freshwater withdrawals; scarcity-weighted water use).

- Land use & land-use change contributions.

- Stage contributions (% share from farm/feed/processing/transport/packaging/retail).


# Business Questions
These are the key business questions to be answered by the end of the project:

1. Which foods are highest/lowest impact by GHG per kg? By kcal? By 100 g protein? 

2. Which lifecycle stages dominate impacts for each food (e.g., farm vs transport vs packaging)? 


3. Top leverage points: which 8–10 foods account for ~80% of total GHG (Pareto) and what stage drives each?

4. Water risk: which foods have extreme scarcity-weighted water use, and where do withdrawals cluster?

5. Dietary swaps: what realistic substitutions (e.g., beef → poultry/legumes; dairy → plant milks) yield the largest impact reduction per serving of protein/kcal?

6. Transport & packaging sensitivity: for which foods are these stages non-trivial (i.e., >10–15%)?

7. Consistency trade-offs: do lower-GHG foods sometimes have higher water or land footprints? What’s the recommended balance?

8. Scenario impact: if a retailer shifts X% of sales from high- to medium-impact foods, what is the projected GHG/water reduction?

# PHASE 2: DATA UNDERSTANDING
The objective of this phase is to load the dataset, understand its structure, and perform initial exploration.

A. Loading and Inspecting the data

In [2]:
import os
import pandas as pd
import zipfile
from tqdm import tqdm

data_root = './data'
csv_files = []

# Step 1: Unzip all files (if needed)
for root, _, files in os.walk(data_root):
    for file in files:
        if file.endswith('.zip'):
            zip_path = os.path.join(root, file)
            extract_path = os.path.splitext(zip_path)[0]
            if not os.path.exists(extract_path):  # Avoid re-extracting
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(extract_path)

# Step 2: Collect CSVs
for root, _, files in os.walk(data_root):
    for file in files:
        if file.lower().endswith('.csv') and '__MACOSX' not in root and not file.startswith('._'):
            csv_files.append(os.path.join(root, file))

print(f"Total CSV files collected: {len(csv_files)}")


Total CSV files collected: 296


Before merging, I'm going to validate column structure across all 60 files to ensure that there are no missing or extra columns, column order is consistent so that I know early if any files need fixing:



In [3]:
from collections import defaultdict

# Step 4: Classify CSVs by column structure
def group_csvs_by_structure(file_list):
    structure_map = defaultdict(list)

    for file in file_list:
        try:
            df_sample = pd.read_csv(file, nrows=5, dtype=str, low_memory=False, encoding='utf-8')
            col_tuple = tuple(df_sample.columns.str.upper().str.strip())
            structure_map[col_tuple].append(file)
        except Exception as e:
            print(f"Skipped {file}: {e}")

    # Report
    print(f"\nFound {len(structure_map)} unique column structures.")
    for i, (cols, files) in enumerate(structure_map.items(), 1):
        print(f"\nStructure {i}: {len(files)} file(s)")
        print("Columns:", cols[:10], "...")  # Show only first 10 columns
        print("Sample file:", files[0])
    return structure_map

structure_map = group_csvs_by_structure(csv_files)



Found 6 unique column structures.

Structure 1: 97 file(s)
Columns: ('TRDTYPE', 'USASTATE', 'DEPE', 'DISAGMOT', 'MEXSTATE', 'CANPROV', 'COUNTRY', 'VALUE', 'SHIPWT', 'FREIGHT_CHARGES') ...
Sample file: ./data\2020\April2020TransBorderRawData\Apr 2020\dot1_0420.csv

Structure 2: 97 file(s)
Columns: ('TRDTYPE', 'USASTATE', 'COMMODITY2', 'DISAGMOT', 'MEXSTATE', 'CANPROV', 'COUNTRY', 'VALUE', 'SHIPWT', 'FREIGHT_CHARGES') ...
Sample file: ./data\2020\April2020TransBorderRawData\Apr 2020\dot2_0420.csv

Structure 3: 96 file(s)
Columns: ('TRDTYPE', 'DEPE', 'COMMODITY2', 'DISAGMOT', 'COUNTRY', 'VALUE', 'SHIPWT', 'FREIGHT_CHARGES', 'DF', 'CONTCODE') ...
Sample file: ./data\2020\April2020TransBorderRawData\Apr 2020\dot3_0420.csv

Structure 4: 2 file(s)
Columns: ('TRDTYPE', 'USASTATE', 'DEPE', 'DISAGMOT', 'MEXSTATE', 'CANPROV', 'COUNTRY', 'DF', 'CONTCODE', 'YEAR') ...
Sample file: ./data\2021\July-to-Dec-2021\New folder\Dec 2021\dot1_2021.csv

Structure 5: 2 file(s)
Columns: ('TRDTYPE', 'USASTATE'

This output shows that there are 6 different/unique columns structure so I'm going to merge each structure separately,and create 3 master DataFrames:

In [4]:
# Map structure indices to group names manually after inspecting them
dot1_files = structure_map[list(structure_map.keys())[0]]
dot2_files = structure_map[list(structure_map.keys())[1]]
dot3_files = structure_map[list(structure_map.keys())[2]]

# If you noticed other variants (e.g. newer versions), you can merge them in too:
# dot1_files += structure_map[list(structure_map.keys())[3]]

def merge_csv_files(file_list, group_name):
    merged = []
    print(f"\nMerging {len(file_list)} files for {group_name}...")

    for file in file_list:
        try:
            chunk = pd.read_csv(file, dtype=str, low_memory=False, encoding='utf-8')
            chunk.columns = chunk.columns.str.upper().str.strip()
            chunk['SOURCE_FILE'] = os.path.basename(file)
            merged.append(chunk)
        except Exception as e:
            print(f"Failed to read {file}: {e}")
    
    df_merged = pd.concat(merged, ignore_index=True)
    print(f"{group_name} merged: shape = {df_merged.shape}")
    return df_merged

dot1 = merge_csv_files(dot1_files, "dot1")
dot2 = merge_csv_files(dot2_files, "dot2")
dot3 = merge_csv_files(dot3_files, "dot3")



Merging 97 files for dot1...
dot1 merged: shape = (8211492, 15)

Merging 97 files for dot2...
dot2 merged: shape = (22505735, 15)

Merging 96 files for dot3...
dot3 merged: shape = (5133293, 13)


Save each of the set as csv file


In [5]:
import pandas as pd
import os

# Create output folder (if not existing)
output_dir = "merged_datasets"
os.makedirs(output_dir, exist_ok=True)

# Function to merge and save each group
def merge_and_save(file_list, group_name):
    merged_data = []
    for file in file_list:
        try:
            df = pd.read_csv(file, dtype=str, low_memory=False, encoding='utf-8')
            df.columns = df.columns.str.upper().str.strip()
            df["SOURCE_FILE"] = os.path.basename(file)
            merged_data.append(df)
        except Exception as e:
            print(f" Failed to read {file}: {e}")

    if merged_data:
        merged_df = pd.concat(merged_data, ignore_index=True)
        output_path = os.path.join(output_dir, f"{group_name}_merged.csv")
        merged_df.to_csv(output_path, index=False)
        print(f" Saved: {output_path} — Shape: {merged_df.shape}")
    else:
        print(f" No valid files to merge for {group_name}")

# Merge and save each group
merge_and_save(dot1_files, "dot1")
merge_and_save(dot2_files, "dot2")
merge_and_save(dot3_files, "dot3")



 Saved: merged_datasets\dot1_merged.csv — Shape: (8211492, 15)
 Saved: merged_datasets\dot2_merged.csv — Shape: (22505735, 15)
 Saved: merged_datasets\dot3_merged.csv — Shape: (5133293, 13)
